In [1]:
import cv2
import numpy as np
import os
import math
import json

In [2]:
# Contour parameters
min_contour_width = 20  
min_contour_height = 20
max_contour_width = 120 
max_contour_height = 120
# Distance from the line
offset = 15
# Lane parameters
lane_1_height_1=160
lane_1_height_2=150
lane_2_height_1=140
lane_2_height_2=130
lane_3_height_1=250
lane_3_height_2=230
lane_1_width_1=210
lane_1_width_2=230
lane_2_width_1=240
lane_2_width_2=260
lane_3_width_1=80
lane_3_width_2=30
matches = []
# Vehicle counters
vehicles_3 = 0
vehicles_2 = 0
# Other helping metrics
frame_number_threshold=15
new_x_1=0
new_y_1=0
new_x_2=0
new_y_2=0

In [3]:
# Line to point distance function
def point_to_line_distance(point, line_start_width, line_start_height, line_end_width, line_end_height):

    A = line_end_height - line_start_height
    B = line_start_width - line_end_width
    C = line_end_width * line_start_height - line_start_width * line_end_height

    distance = abs(A * point[0] + B * point[1] + C) / np.sqrt(A**2 + B**2)
    return distance

In [4]:
# Point to point distance function
def distance_between_points(x1, y1, x2, y2):
    # Calculate the distance using the Euclidean distance formula
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

In [5]:
# Getting centroid
def get_centrolid(x, y, w, h):
   x1 = int(w / 2)
   y1 = int(h / 2)
 
   cx = x + x1
   cy = y + y1
   return cx, cy

In [6]:
# Loading videos
files=[]
folder_path = os.getcwd()
search_string="MaVI18"
for filename in os.listdir(folder_path):
    if search_string in filename:
        files.append(filename)

In [ ]:
results={}

# Iterating through videos

for video in files[]:

    # Defining counters

    last_1=""
    last_2=""
    last_3=""

    vehicles_1 = 0
    vehicles_2 = 0
    vehicles_3 = 0

    motorbikes_1=0
    motorbikes_2=0
    motorbikes_3=0

    cars_1=0
    cars_2=0
    cars_3=0

    transporters_1=0
    transporters_2=0
    transporters_3=0

    trucks_1=0
    trucks_2=0
    trucks_3=0

    motorbikes_1_list=[]
    motorbikes_2_list=[]
    motorbikes_3_list=[]

    cars_1_list=[]
    cars_2_list=[]
    cars_3_list=[]

    transporters_1_list=[]
    transporters_2_list=[]
    transporters_3_list=[]

    trucks_1_list=[]
    trucks_2_list=[]
    trucks_3_list=[]

    # Loading videos
    
    cap = cv2.VideoCapture(video)
    
    cap.set(3, 620)
    cap.set(4, 480)
     
    if cap.isOpened():
       ret, frame1 = cap.read()
    else:
       ret = False
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    
    frame_nr=0
    frame_1_counter=0
    frame_2_counter=0
    frame_3_counter=0
    
    while ret:
    
       frame_1_counter+=1
       frame_2_counter+=1
       frame_3_counter+=1

       # Frame difference calculations and preprocessing
    
       d = cv2.absdiff(frame1, frame2)
       grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
     
       blur = cv2.GaussianBlur(grey, (5, 5), 0)
     
       ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
       dilated = cv2.dilate(th, np.ones((3, 3)))
       kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
     
      
       closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
       contours, h = cv2.findContours(
           closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
       for(i, c) in enumerate(contours):
           (x, y, w, h) = cv2.boundingRect(c)
           contour_valid = (w >= min_contour_width) and (h >= min_contour_height) and (h <= max_contour_height) and (w <= max_contour_width)
     
           if not contour_valid:
               continue

           # Drawing rectangles over centroids
           
           cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)

           # Drawing lines
     
           cv2.line(frame1, (lane_1_width_1, lane_1_height_1), (lane_1_width_2, lane_1_height_2), (0, 255, 0), 5)
           cv2.line(frame1, (lane_2_width_1, lane_2_height_1), (lane_2_width_2, lane_2_height_2), (0, 255, 0), 5)
           cv2.line(frame1, (lane_3_width_1, lane_3_height_1), (lane_3_width_2, lane_3_height_2), (0, 255, 0), 5)
           centrolid = get_centrolid(x, y, w, h)
           matches.append(centrolid)
           cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
           cx, cy = get_centrolid(x, y, w, h)

           # Checking if each point in within the given distance from the line, on right width and classifying vehicle type
           
           for (x, y) in matches:
               if distance_between_points(x, y, new_x_2, new_y_2)>15 and x < lane_1_width_2+3 and x > lane_1_width_1-3 and point_to_line_distance((x, y), lane_1_width_1, lane_1_height_1, lane_1_width_2, lane_1_height_2)<offset:
                    if frame_1_counter<=frame_number_threshold-2 and frame_1_counter>10:
                       if last_1=="motorbike":
                           motorbikes_1-=1
                           try:
                               motorbikes_1_list.pop()
                           except:
                               pass
                       if last_1=="car":
                           cars_1-=1
                           try:
                               cars_1_list.pop()
                           except:
                               pass
                       if last_1=="transporter":
                           transporters_1-=1
                           try:
                               transporters_1_list.pop()
                           except:
                               pass
                       if last_1=="truck":
                           trucks_1-=1
                           try:
                               trucks_1_list.pop()
                           except:
                               pass
                       trucks_1+=1
                       trucks_1_list.append(frame_nr)
                       frame_1_counter=0
                    elif frame_1_counter<=10 and frame_1_counter>6:
                       if last_1=="motorbike":
                           motorbikes_1-=1
                           try:
                               motorbikes_1_list.pop()
                           except:
                               pass
                       if last_1=="car":
                           cars_1-=1
                           try:
                               cars_1_list.pop()
                           except:
                               pass
                       if last_1=="transporter":
                           transporters_1-=1
                           try:
                               transporters_1_list.pop()
                           except:
                               pass
                       if last_1=="truck":
                           trucks_1-=1
                           try:
                               trucks_1_list.pop()
                           except:
                               pass
                       transporters_1+=1
                       transporters_1_list.append(frame_nr)
                       frame_1_counter=0
                    elif frame_1_counter>frame_number_threshold:
                       if w<=30 and h<=30:
                           motorbikes_1+=1
                           motorbikes_1_list.append(frame_nr)
                           last_1="motorbike"
                       elif w>30 and h>30 and w<80 and h<80:
                           cars_1+=1
                           cars_1_list.append(frame_nr)
                           last_1="car"
                       elif w<100 and h<100 and w>=80 and h>=80:
                           transporters_1+=1
                           transporters_1_list.append(frame_nr)
                           last_1="transporter"
                       elif w>=100 and h>=100:
                           trucks_1+=1
                           trucks_1_list.append(frame_nr)
                           last_1="truck"
                       else:
                           cars_1+=1
                           cars_1_list.append(frame_nr)
                           last_1="car"
                       vehicles_1+=1
                       frame_1_counter=0
                    try:
                       matches.remove((x, y))
                    except:
                       pass
               if x > lane_2_width_1-3 and x < lane_2_width_2+3 and point_to_line_distance((x, y), lane_2_width_1, lane_2_height_1, lane_2_width_2, lane_2_height_2)<offset+29:
                    if frame_2_counter<=frame_number_threshold+35 and frame_2_counter>42:
                       if last_2=="motorbike":
                           motorbikes_2-=1
                           try:
                               motorbikes_2_list.pop()
                           except:
                               pass
                       if last_2=="car":
                           cars_2-=1
                           try:
                               cars_2_list.pop()
                           except:
                               pass
                       if last_2=="transporter":
                           transporters_2-=1
                           try:
                               transporters_2_list.pop()
                           except:
                               pass
                       if last_2=="truck":
                           trucks_2-=1
                           try:
                               trucks_2_list.pop()
                           except:
                               pass
                       frame_2_counter=0
                       trucks_2+=1
                       trucks_2_list.append(frame_nr)
                    elif frame_2_counter<=42 and frame_2_counter>35:
                       if last_2=="motorbike":
                           motorbikes_2-=1
                           try:
                               motorbikes_2_list.pop()
                           except:
                               pass
                       if last_2=="car":
                           cars_2-=1
                           try:
                               cars_2_list.pop()
                           except:
                               pass
                       if last_2=="transporter":
                           transporters_2-=1
                           try:
                               transporters_2_list.pop()
                           except:
                               pass
                       if last_2=="truck":
                           trucks_2-=1
                           try:
                               trucks_2_list.pop()
                           except:
                               pass
                       transporters_2+=1
                       transporters_2_list.append(frame_nr)
                       frame_2_counter=0
                    elif frame_2_counter>frame_number_threshold:
                       if w<=23 and h<=23:
                           motorbikes_2+=1
                           motorbikes_2_list.append(frame_nr)
                           last_2="motorbike"
                       elif w>23 and h>23 and w<80 and h<80:
                           cars_2+=1
                           cars_2_list.append(frame_nr)
                           last_2="car"
                       elif w<100 and h<100 and w>=80 and h>=80:
                           transporters_2+=1
                           transporters_2_list.append(frame_nr)
                           last_2="transporter"
                       elif w>=100 and h>=100:
                           trucks_2+=1
                           trucks_2_list.append(frame_nr)
                           last_2="truck"
                       else:
                           cars_2+=1
                           cars_2_list.append(frame_nr)
                           last_2="car"
                       frame_2_counter=0
                       vehicles_2+=1
                    try:
                       matches.remove((x, y))
                    except:
                       pass
               if x < lane_3_width_1 and x > lane_3_width_2 and point_to_line_distance((x, y), lane_3_width_1, lane_3_height_1, lane_3_width_2, lane_3_height_2)<offset:
                    if frame_3_counter<=frame_number_threshold+3 and frame_3_counter>15:
                       if last_3=="motorbike":
                           motorbikes_3-=1
                           try:
                               motorbikes_3_list.pop()
                           except:
                               pass
                       if last_3=="car":
                           cars_3-=1
                           try:
                               cars_3_list.pop()
                           except:
                               pass
                       if last_3=="transporter":
                           transporters_3-=1
                           try:
                               transporters_3_list.pop()
                           except:
                               pass
                       if last_3=="truck":
                           trucks_3-=1
                           try:
                               trucks_3_list.pop()
                           except:
                               pass
                       trucks_3+=1
                       trucks_3_list.append(frame_nr)
                       frame_3_counter=0
                    elif frame_3_counter<=15 and frame_3_counter>12:
                       if last_3=="motorbike":
                           motorbikes_3-=1
                           try:
                               motorbikes_3_list.pop()
                           except:
                               pass
                       if last_3=="car":
                           cars_3-=1
                           try:
                               cars_3_list.pop()
                           except:
                               pass
                       if last_3=="transporter":
                           transporters_3-=1
                           try:
                               transporters_3_list.pop()
                           except:
                               pass
                       if last_3=="truck":
                           trucks_3-=1
                           try:
                               trucks_3_list.pop()
                           except:
                               pass
                       transporters_3+=1
                       transporters_3_list.append(frame_nr)
                       frame_3_counter=0
                    elif frame_3_counter>frame_number_threshold:
                       if w<=30 and h<=30:
                           motorbikes_3+=1
                           motorbikes_3_list.append(frame_nr)
                           last_3="motorbike"
                       elif w>30 and h>30 and w<80 and h<80:
                           cars_3+=1
                           cars_3_list.append(frame_nr)
                           last_3="car"
                       elif w<100 and h<100 and w>=80 and h>=80:
                           transporters_3+=1
                           transporters_3_list.append(frame_nr)
                           last_3="transporter"
                       elif w>=100 and h>=100:
                           trucks_3+=1
                           trucks_3_list.append(frame_nr)
                           last_3="truck"
                       else:
                           cars_3+=1
                           cars_3_list.append(frame_nr)
                           last_3="car"
                       frame_3_counter=0
                       vehicles_3+=1
                    try:
                       matches.remove((x, y))
                    except:
                       pass
       # Text with current counts on the video
     
       cv2.putText(frame1, f"Total Vehicle Detected Lane 1: {vehicles_1}", (0, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_1}, cars: {cars_1}, transporters: {transporters_1}, trucks: {trucks_1}", (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
       cv2.putText(frame1, f"Total Vehicle Detected Lane 2: {vehicles_2}", (0, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_2}, cars: {cars_2}, transporters: {transporters_2}, trucks: {trucks_2}", (0, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
       cv2.putText(frame1, f"Total Vehicle Detected Lane 3: {vehicles_3}", (0, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_3}, cars: {cars_3}, transporters: {transporters_3}, trucks: {trucks_3}", (0, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
       cv2.imshow("Vehicle Detection", frame1)
        
       if cv2.waitKey(1) == 27:
           break
       # Wait for a specified amount of time
       if cv2.waitKey(1) & 0xFF == ord('q'):
            break
       frame1 = frame2
       ret, frame2 = cap.read()
       frame_nr+=1
     
    cv2.destroyAllWindows()
    cap.release()

    # Saving results into dictionary
    
    results[video]={"motorbikes_lane_1":motorbikes_1, "motorbikes_lane_1_frames":motorbikes_1_list, "cars_lane_1":cars_1, "cars_lane_1_frames":cars_1_list, "transporters_lane_1":transporters_1, "transporters_lane_1_frames":transporters_1_list, "trucks_lane_1":trucks_1, "trucks_lane_1_frames":trucks_1_list,
    "motorbikes_lane_2":motorbikes_2, "motorbikes_lane_2_frames":motorbikes_2_list, "cars_lane_2":cars_2, "cars_lane_2_frames":cars_2_list, "transporters_lane_2":transporters_2, "transporters_lane_2_frames":transporters_2_list, "trucks_lane_2":trucks_2, "trucks_lane_2_frames":trucks_2_list,
    "motorbikes_lane_3":motorbikes_3, "motorbikes_lane_3_frames":motorbikes_3_list, "cars_lane_3":cars_3, "cars_lane_3_frames":cars_3_list, "transporters_lane_3":transporters_3, "transporters_lane_3_frames":transporters_3_list, "trucks_lane_3":trucks_3, "trucks_lane_3_frames":trucks_3_list}

In [ ]:
# Saving into json file

with open(f"{search_string}.json", "w") as json_file:
    json.dump(results, json_file)